# 第08章：RAG基础（Retrieval Augmented Generation）

## 学习目标

本章将学习：
1. 理解RAG的核心概念和工作原理
2. 掌握2-Step RAG的实现
3. 使用Retriever接口
4. 构建完整的RAG Chain
5. 理解RAG vs 传统问答的区别

## 什么是RAG？

RAG（Retrieval Augmented Generation，检索增强生成）是一种结合检索和生成的技术：

1. **检索（Retrieval）**：从知识库中检索相关文档
2. **增强（Augmented）**：将检索到的文档作为上下文
3. **生成（Generation）**：LLM基于上下文生成答案

### RAG的优势

- 减少幻觉：答案基于真实文档
- 知识更新：无需重新训练模型
- 可追溯：可以查看引用来源
- 成本效益：比微调模型更经济


In [3]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

# 配置Embedding模型
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL
)

print("环境配置完成！")
print(f"Chat模型: {model.model_name}")
print(f"Embedding模型: {embeddings.model}")

环境配置完成！
Chat模型: gpt-4.1-mini
Embedding模型: text-embedding-3-small


## 2. RAG工作流程

### RAG的两种主要模式

#### 2-Step RAG（两步RAG）
- 总是先检索，再生成
- 流程固定：查询 → 检索 → 生成
- 每次查询只需1次LLM调用
- 快速高效，适合简单场景

#### Agentic RAG（智能体RAG）
- Agent决定何时检索
- 可以多次检索
- 更灵活但需要更多token
- 适合复杂推理场景

本章重点学习2-Step RAG。

In [6]:
### 2.1 准备知识库

print("【准备知识库】")
print()

# 创建知识库文档
documents = [
    Document(
        page_content="LangChain是一个用于构建LLM应用的开源框架，支持多种模型提供商。",
        metadata={"source": "doc1", "topic": "LangChain"}
    ),
    Document(
        page_content="RAG（检索增强生成）通过检索外部知识来增强LLM的回答能力。",
        metadata={"source": "doc2", "topic": "RAG"}
    ),
    Document(
        page_content="Vector Store用于存储文档的向量表示，支持语义搜索。",
        metadata={"source": "doc3", "topic": "技术"}
    ),
    Document(
        page_content="LCEL（LangChain Expression Language）是LangChain的核心语法，使用管道操作符连接组件。",
        metadata={"source": "doc4", "topic": "LCEL"}
    ),
    Document(
        page_content="Agent可以自主决策使用哪些工具来完成任务，是LangChain的高级特性。",
        metadata={"source": "doc5", "topic": "Agent"}
    ),
]

# 创建Vector Store并索引
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents)

print(f"知识库已创建，包含 {len(documents)} 个文档")
print("文档主题:", set(doc.metadata["topic"] for doc in documents))

【准备知识库】



知识库已创建，包含 5 个文档
文档主题: {'技术', 'Agent', 'LCEL', 'RAG', 'LangChain'}


## 3. Retriever接口

### 什么是Retriever？

Retriever是一个Runnable接口，用于检索文档。它比Vector Store更通用：
- Vector Store只能基于向量相似度检索
- Retriever可以实现各种检索策略
- Retriever是Runnable，可以用`|`组合

### 核心方法

- `invoke(query)`: 同步检索
- `ainvoke(query)`: 异步检索
- `batch(queries)`: 批量检索

In [7]:
### 3.1 创建Retriever

print("【创建Retriever】")
print()

# 从Vector Store创建Retriever
retriever = vector_store.as_retriever(
    search_type="similarity",  # 相似度搜索
    search_kwargs={"k": 2}      # 返回top-2结果
)

print("Retriever已创建")
print(f"搜索类型: similarity")
print(f"返回结果数: 2")
print()

# 测试检索
query = "什么是RAG？"
docs = retriever.invoke(query)

print(f"查询: {query}")
print(f"检索到 {len(docs)} 个文档:")
for i, doc in enumerate(docs, 1):
    print(f"\n{i}. {doc.page_content}")
    print(f"   来源: {doc.metadata['source']}")

【创建Retriever】

Retriever已创建
搜索类型: similarity
返回结果数: 2

查询: 什么是RAG？
检索到 2 个文档:

1. RAG（检索增强生成）通过检索外部知识来增强LLM的回答能力。
   来源: doc2

2. LangChain是一个用于构建LLM应用的开源框架，支持多种模型提供商。
   来源: doc1


In [8]:
### 3.2 批量检索

print("【批量检索】")
print()

# 批量检索多个查询
queries = [
    "LangChain是什么？",
    "什么是Agent？",
    "LCEL的作用是什么？"
]

# 使用batch方法
results = retriever.batch(queries)

for query, docs in zip(queries, results):
    print(f"查询: {query}")
    print(f"  最相关: {docs[0].page_content[:50]}...")
    print()

【批量检索】

查询: LangChain是什么？
  最相关: LangChain是一个用于构建LLM应用的开源框架，支持多种模型提供商。...

查询: 什么是Agent？
  最相关: Agent可以自主决策使用哪些工具来完成任务，是LangChain的高级特性。...

查询: LCEL的作用是什么？
  最相关: LCEL（LangChain Expression Language）是LangChain的核心语法...



## 4. 构建2-Step RAG Chain

### RAG Chain的核心步骤

1. **检索（Retrieve）**：根据查询检索相关文档
2. **生成（Generate）**：将文档作为上下文，LLM生成答案

### 使用LCEL构建RAG Chain

LCEL让我们可以用管道操作符`|`优雅地组合组件。

In [9]:
### 4.1 简单RAG Chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("【构建简单RAG Chain】")
print()

# 定义格式化函数
def format_docs(docs):
    """将文档列表格式化为字符串"""
    return "\n\n".join(doc.page_content for doc in docs)

# 创建提示词模板
template = """基于以下上下文回答问题。如果上下文中没有相关信息，请说"我不知道"。

上下文：
{context}

问题：{question}

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 构建RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print("RAG Chain已构建")
print("组件: Retriever → Format → Prompt → Model → Parser")
print()

# 测试
question = "什么是RAG？"
answer = rag_chain.invoke(question)

print(f"问题: {question}")
print(f"回答: {answer}")

【构建简单RAG Chain】

RAG Chain已构建
组件: Retriever → Format → Prompt → Model → Parser

问题: 什么是RAG？
回答: RAG（检索增强生成）是通过检索外部知识来增强大型语言模型（LLM）回答能力的一种方法。


In [10]:
### 4.2 多问题测试

print("【测试多个问题】")
print()

test_questions = [
    "LangChain支持哪些功能？",
    "LCEL是什么？",
    "Agent的作用是什么？",
    "如何训练大型语言模型？"  # 知识库中没有的问题
]

for question in test_questions:
    answer = rag_chain.invoke(question)
    print(f"Q: {question}")
    print(f"A: {answer}")
    print("-" * 60)
    print()

【测试多个问题】

Q: LangChain支持哪些功能？
A: LangChain支持构建LLM应用，支持多种模型提供商，并且具备Agent功能，能够自主决策使用哪些工具来完成任务。
------------------------------------------------------------

Q: LCEL是什么？
A: LCEL（LangChain Expression Language）是LangChain的核心语法，使用管道操作符连接组件。
------------------------------------------------------------

Q: Agent的作用是什么？
A: Agent的作用是能够自主决策使用哪些工具来完成任务。
------------------------------------------------------------

Q: 如何训练大型语言模型？
A: 我不知道。
------------------------------------------------------------



## 5. 带来源引用的RAG

在实际应用中，我们通常希望返回答案的同时，也返回引用的文档来源。

In [12]:
### 5.1 返回答案和来源

from langchain_core.runnables import RunnableParallel

print("【带来源的RAG】")
print()

# 构建带来源的RAG Chain
rag_chain_with_source = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
).assign(
    answer=lambda x: (prompt | model | StrOutputParser()).invoke({
        "context": format_docs(x["context"]),
        "question": x["question"]
    })
)

# 测试
question = "什么是LCEL？"
result = rag_chain_with_source.invoke(question)

print(f"问题: {question}")
print(f"\n答案: {result['answer']}")
print(f"\n引用来源:")
for i, doc in enumerate(result['context'], 1):
    print(f"{i}. [{doc.metadata['source']}] {doc.page_content[:60]}...")

【带来源的RAG】

问题: 什么是LCEL？

答案: LCEL（LangChain Expression Language）是LangChain的核心语法，使用管道操作符连接组件。

引用来源:
1. [doc4] LCEL（LangChain Expression Language）是LangChain的核心语法，使用管道操作符连接...
2. [doc1] LangChain是一个用于构建LLM应用的开源框架，支持多种模型提供商。...


## 6. 实战项目：完整的RAG问答系统

构建一个功能完整的RAG系统，包括：
1. 文档加载和索引
2. 智能检索
3. 答案生成
4. 来源追踪

In [16]:
print("【完整RAG系统】")
print()

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

class RAGSystem:
    """完整的RAG问答系统"""
    
    def __init__(self, embeddings, model):
        self.embeddings = embeddings
        self.model = model
        self.vector_store = InMemoryVectorStore(embeddings)
        self.retriever = None
        
    def load_and_index(self, file_path, chunk_size=200, chunk_overlap=50):
        """加载文档并建立索引"""
        # 加载文档
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        
        # 分割文档
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        splits = splitter.split_documents(docs)
        
        # 索引
        self.vector_store.add_documents(splits)
        
        # 创建retriever
        self.retriever = self.vector_store.as_retriever(
            search_kwargs={"k": 3}
        )
        
        return len(splits)
    
    def create_rag_chain(self):
        """创建RAG链"""
        template = """基于以下上下文回答问题。请简洁明了地回答。

上下文：
{context}

问题：{question}

回答："""
        
        prompt = ChatPromptTemplate.from_template(template)
        
        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)
        
        answer_chain = prompt | self.model | StrOutputParser()
        
        # 构建链
        self.rag_chain = RunnableParallel(
            {
                "context": self.retriever,
                "question": RunnablePassthrough()
            }
        ).assign(
            answer=lambda x: answer_chain.invoke({
                "context": format_docs(x["context"]),
                "question": x["question"]
            })
        )
    
    def ask(self, question):
        """提问并返回答案和来源"""
        result = self.rag_chain.invoke(question)
        return {
            "question": question,
            "answer": result["answer"],
            "sources": result["context"]
        }

# 创建RAG系统
rag_system = RAGSystem(embeddings, model)

# 加载文档
num_chunks = rag_system.load_and_index("../data/langchain_intro.txt")
print(f"已索引 {num_chunks} 个文档块")

# 创建RAG链
rag_system.create_rag_chain()
print("RAG系统已就绪")
print()

# 测试问答
test_questions = [
    "LangChain的主要特性有哪些？",
    "LangChain支持哪些模型提供商？",
    "什么是RAG工具链？"
]

print("=" * 60)
for question in test_questions:
    result = rag_system.ask(question)
    print(f"\nQ: {result['question']}")
    print(f"A: {result['answer']}")
    print(f"\n参考文档数: {len(result['sources'])}")
    print("=" * 60)

print("\nRAG系统演示完成！")

【完整RAG系统】

已索引 3 个文档块
RAG系统已就绪


Q: LangChain的主要特性有哪些？
A: LangChain的主要特性包括：  
1. 模型集成，支持多种语言模型提供商；  
2. 强大的提示词模板系统，支持动态变量、条件逻辑和少样本学习；  
3. 链式调用，通过LCEL构建复杂处理流程；  
4. 完整的RAG工具链支持，包括文档加载、文本分割和向量数据库集成；  
5. Agent框架，使模型能够自主决策和使用工具。

参考文档数: 3

Q: LangChain支持哪些模型提供商？
A: LangChain支持OpenAI、Anthropic、Google等多种语言模型提供商。

参考文档数: 3

Q: 什么是RAG工具链？
A: RAG工具链是指LangChain内置的一整套支持检索增强生成（Retrieval-Augmented Generation）的工具，包括文档加载器、文本分割器和向量数据库集成等，用于实现基于外部知识的智能生成。

参考文档数: 3

RAG系统演示完成！


## 7. RAG vs 传统问答

对比使用RAG和不使用RAG的区别

In [17]:
print("【RAG vs 传统问答对比】")
print()

question = "LangChain支持哪些Agent框架？"

# 1. 不使用RAG（直接问模型）
print("1. 不使用RAG（直接问模型）:")
direct_answer = model.invoke(question)
print(f"   {direct_answer.content[:150]}...")
print()

# 2. 使用RAG
print("2. 使用RAG（基于知识库）:")
rag_result = rag_system.ask(question)
print(f"   {rag_result['answer']}")
print()

print("对比分析:")
print("- 不使用RAG: 可能产生幻觉，信息可能过时")
print("- 使用RAG: 基于实际文档，答案更准确可靠")


【RAG vs 传统问答对比】

1. 不使用RAG（直接问模型）:
   截至目前，LangChain 支持多种 Agent 框架，主要包括以下几种：

1. **Zero-Shot Agent**  
   通过提示模板直接让模型根据输入决定调用哪些工具，无需示例演示。

2. **Conversational Agent**  
   适用于对话场景，能够结合上下文进...

2. 使用RAG（基于知识库）:
   LangChain提供的Agent框架，使模型能够自主决策和使用工具，支持问答系统、文档分析、代码生成、对话机器人、数据提取和智能搜索等应用场景。

对比分析:
- 不使用RAG: 可能产生幻觉，信息可能过时
- 使用RAG: 基于实际文档，答案更准确可靠


## 8. 总结与最佳实践

### 核心概念回顾

1. RAG = 检索 + 生成
2. 2-Step RAG: 固定流程，快速高效
3. Retriever: Runnable接口，支持各种检索策略
4. RAG Chain: 使用LCEL优雅组合组件

### RAG的关键组件

- Document Loader: 加载文档
- Text Splitter: 分割文档
- Embeddings: 文本向量化
- Vector Store: 存储向量
- Retriever: 检索接口
- Prompt Template: 构建提示词
- LLM: 生成答案

### 最佳实践

1. 文档质量: 确保文档内容准确、完整
2. 分割策略: 合理设置chunk_size和overlap
3. 检索数量: k值通常设置为3-5
4. 提示词设计: 明确指示模型基于上下文回答
5. 来源追踪: 返回引用文档便于验证

### 常见问题

- 检索不到相关文档: 调整k值或改进查询
- 答案不准确: 优化提示词或提高文档质量
- 响应慢: 减少k值或优化向量存储
- 成本高: 使用更小的embedding模型